In [1]:
import gensim, logging
import os
from gensim.models.word2vec import PathLineSentences, LineSentence
import pandas as pd
import imp
from tqdm import tqdm_notebook
import multiprocessing
import numpy as np

In [2]:
import sys
sys.path.append('/Users/ak/Yandex.Disk.localized/sentiment-neural_past_from_Denis_Kirjanov/src')

In [3]:
import transformer
imp.reload(transformer)
from nltk_sent_tokenize_binding import NLTKSentenceTokenizer

Подготовить тексты из чатов:
* Выгрузить тексты
* Выгрузить модель
* Нормализовать
* Выгрузить ранее нормализованные тексты
* Объединить
* Сохранить в один файл

In [9]:
root_path = '/Users/ak/Yandex.Disk.localized/Магистратура ВШЭ/Chat_word_autofill/'
df = pd.read_csv(root_path + 'Client_messages_cleaned.csv', encoding='cp1251', delimiter=';', header=None)
df.head()

,0,1
0,2,"Я внес деньги на карту. Когда они придут, я их..."
1,4,Кредитная
2,6,По карте
3,8,Да
4,10,Да


In [14]:
false_stoppings = ["лат", "греч", "проц", "млн", "млрд", "итал", "фр", "рус", "кв", "м", "тыс", "руб", "англ", "п",
                       "ст", "гор", "обл", "пос", "д", "дер", "др", "пр", "рег", "ул", "к", "т.е", "т.п", "юр", "физ"]
sentence_tokenizer = NLTKSentenceTokenizer(false_stoppings)

In [19]:
sentences = []
for text in tqdm_notebook(df[1].values):
    if type(text) in [str, object]:
        for sentence in text.split('\n'):
            s = sentence_tokenizer(sentence)
            sentences.extend(s)

HBox(children=(IntProgress(value=0, max=819752), HTML(value='')))

In [20]:
len(sentences)

1028955

In [21]:
import pickle
with open('model_tokenize.pkl', 'rb') as f:
    model = pickle.load(f)

In [27]:

def norm12345(s):
    tokens = model.lemmatizer.process(s)
    return ' '.join(tokens)

In [28]:
%%time
pool = multiprocessing.Pool(3)
sent_norm = pool.map(norm12345, sentences)

CPU times: user 880 ms, sys: 533 ms, total: 1.41 s
Wall time: 1min 51s


In [41]:
sentences_prev = pd.read_csv('sentences_normalized.txt', header=None)

In [42]:
sentences_prev.head()

,0
0,рабочий место выделять рабочий место кредит касса
1,pos терминал сенсорный экран для управление сб...
2,очередь иметься
3,free wi fi tattelecom good while waiting
4,so pretty interior though


In [ ]:
sent_norm.extend(sentences_prev[0].values)

In [45]:
len(sent_norm)

1586241

In [46]:
np.savetxt('sentences_normalized_with_chats.txt', sent_norm, fmt='%s')

In [47]:
%%time

ver = 2
logging.basicConfig(filename='w2v_normv' + str(ver) + '_train.log', format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)


# class StringsFeeder(object):
#     def __init__(self, dirname):
#         self.dirname = dirname
#         self.files = [f for f in os.listdir(self.dirname) if os.path.join(self.dirname, f) and f.endswith('.txt')]
#         print(self.files)

#     def __iter__(self):
#         for fname in self.files:
#             for line in open(os.path.join(self.dirname, fname), encoding='utf8'):
#                 yield line.split()

sentences = LineSentence(root_path + 'sentences_normalized_with_chats.txt')



CPU times: user 32 µs, sys: 89 µs, total: 121 µs
Wall time: 124 µs


In [48]:
%%time
model = gensim.models.Word2Vec(sentences, min_count=20, size=300, workers=3, iter=40, window=4)
model.save(root_path + '/model_normalized_with_chats_' + str(ver) + '.bin')

CPU times: user 16min 19s, sys: 8.52 s, total: 16min 28s
Wall time: 8min 39s


In [51]:
emb = model.wv.get_keras_embedding()

Using TensorFlow backend.
/Users/ak/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [58]:
model.wv.vocab['спать']

In [ ]:
def get_vector(word):
    try:
        vec = model.wv(word)
        return vec
    except KeyError:
        return np.zeros(model.wv.vector_size)

In [59]:
model.wv.vector_size

300

In [4]:
X = np.random.uniform(size=(100, 6)) > 0.7

In [6]:
X = X.astype('int32')

In [7]:
X

array([[0, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [1, 0, 1, 0, 0, 1],
       [1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1],
       [1, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 1],
       [1, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 1],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
 